In [1]:
import pandas as pd
from openai import OpenAI
from datetime import datetime
import json

In [2]:
# Initialize OpenAI client with your API key
import openai
openai.api_key = "Yourkeyhere"
client = openai.OpenAI(api_key="Yourkeyhere")


In [3]:
# Load template and input Excel files
#template = pd.ExcelFile('template.xlsx')
input_file_name = "Test.xlsx"
output_file_name = "Done-Test.xlsx"
input_data = pd.read_excel(input_file_name, sheet_name=None)  # read all sheets into a dict of DataFrames
print("File extraction done")

File extraction done


In [4]:
# Process Cookies sheet
cookies_df = input_data.get('Cookies')

#Ensure required columns exist
for col in ['Category', 'Description']:
    if col not in cookies_df.columns:
        cookies_df[col] = ''

print(cookies_df.head())

                Domain                                       Name  \
0  avatar.deloitte.com                          disclaimer_agreed   
1  avatar.deloitte.com                                       s_nr   
2         deloitte.com                     TEST_AMCV_COOKIE_WRITE   
3         deloitte.com   AMCV_42A70C3A53DADE170A490D4D%40AdobeOrg   
4         deloitte.com  AMCVS_42A70C3A53DADE170A490D4D%40AdobeOrg   

                                               Value  \
0                                               True   
1                                  1750926150470-New   
2                                                NaN   
3  179643557%7CMCIDTS%7C20266%7CMCMID%7C588699990...   
4                                               True   

                 Expiration   Lifespan  ThirdParty Path  Secure  HTTP Only  \
0  2025-07-26T08:22:24.000Z  Permanent       False    /   False      False   
1                  Infinity    Session       False    /   False      False   
2  1970-01-01T

In [5]:
import time

cookies_df['Description'] = cookies_df['Description'].astype(str)
cookies_df['Category'] = cookies_df['Category'].astype(str)

# Iterate through each row in Cookies sheet
for i, row in cookies_df.iterrows():
    name = row['Name']  # get the cookie name
    expiration = row['Expiration']  # get expiration value
    lifespan = row['Lifespan']  # get lifespan value
    # Check and format expiration date or lifespan
    if expiration == 'Infinity':
            # If expiration is Infinity, set lifespan as Session
            cookies_df.at[i, 'Lifespan'] = 'Session'    
    elif pd.notnull(expiration) and expiration != 'Infinity': 
        # Format valid date to required ISO format
        expiration = str(expiration).strip()  # remove spaces
        expiration = expiration.replace('/', '-')  # replace slashes with dashes if needed
        expiration = pd.to_datetime(expiration,errors='coerce').strftime('%Y-%m-%dT%H:%M:%S.000Z')    
        print("log:",expiration)
        cookies_df.at[i, 'Expiration'] = expiration
        cookies_df.at[i, 'Lifespan'] = 'Permanent'
    elif lifespan == 'Session':
        # If lifespan is Session, set the expiration as Infinity
        cookies_df.at[i, 'Expiration'] = 'Infinity'
# Call ChatGPT to categorize cookie and generate description
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a cookie categorizer."},
            {"role": "user", "content": f"Return JSON object with keys 'Category' and 'Description' for cookie name {name} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {name}"}],
        temperature=0 )
    text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
    print("GPT:",text)
    result = json.loads(text)  # parse JSON
    cookies_df.at[i, 'Category'] = result.get('Category', '').strip()
    cookies_df.at[i, 'Description'] = result.get('Description', '').strip()
    sucess = True #done!
    print(f"Got the category done")

log: 2025-07-26T08:22:24.000Z
GPT: {
    "Category": "Strictly Necessary Cookies",
    "Description": "Required to track user agreement with website disclaimers."
}
Got the category done
GPT: {
    "Category": "Analytics and Performance Cookies",
    "Description": "Used by Adobe Analytics to identify unique visitors."
}
Got the category done
log: 1970-01-01T00:00:01.000Z
GPT: {
    "Category": "Analytics and Performance Cookies",
    "Description": "Used to track user behavior and measure website performance."
}
Got the category done
log: 2027-06-26T08:22:19.000Z
GPT: {
    "Category": "Analytics and Performance Cookies",
    "Description": "Used by Adobe Analytics to collect anonymous data on user behavior and website interactions."
}
Got the category done
GPT: {
    "Category": "Analytics and Performance Cookies",
    "Description": "This cookie is used to collect information about how visitors use the website for analytics purposes."
}
Got the category done
log: 1970-01-01T00:00:00

In [13]:
import openpyxl
# Find lowest expiration timestamp

# Extract the expiration dates after 2000 year
cookies_df['Expiration'] = pd.to_datetime(cookies_df['Expiration'],errors = 'coerce')
valid_dates = cookies_df[cookies_df['Expiration'].dt.year>=2000]['Expiration']
print(f"all dates:{valid_dates}")

# Find the lowest year to find the cookie scan date
min_date = valid_dates.min()
print(f"Minimum date is:{min_date}")

# convert the lowest date into iSO standards
manual_scan_date = min_date.strftime('%Y-%m-%dT%H:%M:%S.000Z')
print(f"manual_scan_date is:{manual_scan_date}")

# Need to update the expiration only for the date.
cookies_df['Expiration'] = cookies_df['Expiration'].where(
    cookies_df['Expiration'].apply(lambda x: isinstance(x,pd.Timestamp)), pd.NaT
).apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S.000Z") if pd.notna(x) else 'Infinity')

# print("DataFrame shape:",cookies_df.shape)
# print("Column names:",cookies_df.columns.tolist())
# print("Last 5 rows:\n",cookies_df.head())

#Avoid duplicating footer rows and Append Timestamp and Website info
if not cookies_df.iloc[-3:, 0].str.contains("Time stamp of Manual Scan run").any():     
    cookies_df.loc[len(cookies_df)] = [''] * len(cookies_df.columns)     
    cookies_df.loc[len(cookies_df)] = ['Time stamp of Manual Scan run', manual_scan_date] + [''] * (len(cookies_df.columns) - 2)     
    domain_value = cookies_df['Domain'].iloc[0] if 'Domain' in cookies_df.columns else ''     
    cookies_df.loc[len(cookies_df)] = ['Website', domain_value] + [''] * (len(cookies_df.columns) - 2) 

# Convert all to string before writing to Excel 
cookies_df = cookies_df.astype(str)

all dates:0   2025-07-26 08:22:24+00:00
3   2027-06-26 08:22:19+00:00
7   2025-12-23 08:22:20+00:00
Name: Expiration, dtype: datetime64[ns, UTC]
Minimum date is:2025-07-26 08:22:24+00:00
manual_scan_date is:2025-07-26T08:22:24.000Z


In [14]:
# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl') as writer:
    cookies_df.to_excel(writer, sheet_name='Cookies', index=False)

print("Done!")

Done!


In [34]:
# Process Local Storage sheet
local_df = input_data.get('Local Storage')
if local_df is not None:
    for i, row in local_df.iterrows():
        key = row['Key']  # key name
 
        # Call ChatGPT to get category and description
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cookie categorizer."},
                {"role": "user", "content": f"Return JSON object with keys 'Category' and 'Description' for local storage cookie name {key} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {key}"}],
            temperature=0 )
        text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
        print("GPT:",text)
        result = json.loads(text)  # parse JSON
        local_df.at[i, 'Category'] = result.get('Category', '').strip()
        local_df.at[i, 'Description'] = result.get('Description', '').strip()
        sucess = True #done!
        print(f"Got the category done for local storage")

GPT: {
  "Category": "Strictly Necessary Cookies",
  "Description": "Essential for user authentication and security purposes."
}
Got the category done for local storage


D:\Users\skotteeswaran\AppData\Local\Temp\ipykernel_2388\1940574995.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Strictly Necessary Cookies' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  local_df.at[i, 'Category'] = result.get('Category', '').strip()
D:\Users\skotteeswaran\AppData\Local\Temp\ipykernel_2388\1940574995.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Essential for user authentication and security purposes.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  local_df.at[i, 'Description'] = result.get('Description', '').strip()


GPT: {
  "Category": "Strictly Necessary Cookies",
  "Description": "Essential for user authentication and security purposes."
}
Got the category done for local storage
GPT: {
  "Category": "Strictly Necessary Cookies",
  "Description": "Stores information related to Microsoft Authentication Library (MSAL) client for authentication purposes."
}
Got the category done for local storage
GPT: {
  "Category": "Strictly Necessary Cookies",
  "Description": "Stores the user's ID token for authentication purposes."
}
Got the category done for local storage
GPT: {
    "Category": "Strictly Necessary Cookies",
    "Description": "Essential for website functionality and security."
}
Got the category done for local storage
GPT: {
    "Category": "Analytics and Performance Cookies",
    "Description": "Collects data on user interaction to improve website performance."
}
Got the category done for local storage
Done!


In [38]:
# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl', mode='a', if_sheet_exists ='replace') as writer:
    local_df.to_excel(writer, sheet_name='Local Storage', index=False)

print("Done!")

Done!


In [ ]:
# Process Session Storage sheet
session_df = input_data.get('Session Storage')
if session_df is not None:
    for i, row in session_df.iterrows():
        key = row['Key']  # key name
 
        # Call ChatGPT to get category and description
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cookie categorizer."},
                {"role": "user", "content": f"Return JSON object with keys 'Category' and 'Description' for local storage cookie name {key} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {key}"}],
            temperature=0 )
        text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
        print("GPT:",text)
        result = json.loads(text)  # parse JSON
        session_df.at[i, 'Category'] = result.get('Category', '').strip()
        session_df.at[i, 'Description'] = result.get('Description', '').strip()
        sucess = True #done!
        print(f"Got the category done for Session storage")

In [ ]:
# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl', mode='a', if_sheet_exists ='replace') as writer:
    session_df.to_excel(writer, sheet_name='Local Storage', index=False)

print("Done!")